In [16]:
import pandas
import os
import sys
import numpy as np
import nibabel as ni
import matplotlib.pyplot as plt
import seaborn as sns
from glob import glob
from scipy import stats
from mpl_toolkits.mplot3d import Axes3D
from nilearn import plotting
from nilearn.datasets import MNI152_FILE_PATH
from sklearn.cross_decomposition import CCA, PLSRegression
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

In [ ]:
wdir = '/data1/users/jvogel/Allen_Human_Brain_Atlas/'
gdf = pandas.read_csv(os.path.join(wdir,'gxp_correlation_wholebrain_results_NEW.csv'))
shtz = sorted(glob(os.path.join(wdir,'normalized_microarray_donor*/MExp_all_genes_ctr_for_donor')))
holder = []
for sht in shtz:
    holder.append(pandas.read_csv(sht,index_col=0))
bigdf = pandas.concat(holder,axis=1)
del(holder)
bigdf = bigdf.values.T

In [18]:
gdf = pandas.read_csv('/Users/jakevogel/Science/AHBA/MAIN_gcx_wholebrain_info_W_DKT_label.csv')
gdf.columns

Index(['Unnamed: 0', 'donor', 'sample', 'structure_id', 'slab_num', 'well_id',
       'slab_type', 'structure_acronym', 'structure_name', 'polygon_id',
       'mri_voxel_x', 'mri_voxel_y', 'mri_voxel_z', 'mni_x', 'mni_y', 'mni_z',
       'gabe_label', 'mni_nlin_x', 'mni_nlin_y', 'mni_nlin_z',
       'AP_axis_gxp_signature_similarity_SPCR', 'surface_coordinate_index',
       'dkt_label', 'dkt_region'],
      dtype='object')

In [19]:
too_few = [x for x in gdf.structure_acronym.value_counts().index if gdf.structure_acronym.value_counts()[x] < 2]
gdf.loc[:,'new_structure_acronym'] = ['Other' if x in too_few else x for x in gdf.structure_acronym.values]

In [2]:
bigdf = pandas.DataFrame(bigdf)
tr,te = train_test_split(bigdf, stratify=gdf.new_structure_acronym.values, random_state=123)
tr_y = gdf.loc[tr.index,['mni_nlin_y','mni_nlin_z']]
te_y = gdf.loc[te.index,['mni_nlin_y','mni_nlin_z']]

In [ ]:
cca_mod = CCA()
cca_mod.fit(tr,tr_y)

In [6]:
plotr = pandas.concat([pandas.Series(cca_mod.predict(te)[:,0], index = te_y.index), 
                       pandas.Series(cca_mod.predict(te)[:,1], index = te_y.index), 
                       pandas.Series(te_y.loc[:,te_y.columns[0]]),
                       pandas.Series(te_y.loc[:,te_y.columns[1]]), 
                       gdf.loc[te_y.index,'slab_type']], 
                      axis=1)
plotr.columns = ['predicted_y', 'predicted_z', 'observed_y', 'observed_z', 'slab_type']

NameError: name 'cca_mod' is not defined

In [ ]:
#sns.set_context('paper')

In [ ]:
for i in range(2):
    plt.close()
    sns.lmplot(plotr.columns[i], plotr.columns[i+2], data = plotr)
    plt.show()
    print(stats.pearsonr(cca_mod.predict(te)[:,i],te_y.loc[:,te_y.columns[i]]))

In [ ]:
for i in range(2):
    plt.close()
    sns.lmplot(plotr.columns[i], plotr.columns[(3-i)], data = plotr)
    plt.show()
    print(stats.pearsonr(cca_mod.predict(te)[:,i],te_y.loc[:,te_y.columns[1-i]]))

In [ ]:
for i in range(2):
    plt.close()
    sns.lmplot(plotr.columns[i], plotr.columns[i+2], col = 'slab_type', data = plotr)
    plt.show()

In [ ]:
for i in range(2):
    plt.close()
    sns.lmplot(plotr.columns[i], plotr.columns[(3-i)], col = 'slab_type', data = plotr)
    plt.show()

In [ ]:
probes = pandas.read_csv('/home/users/jvogel/Science/Allen_Human_Brain_Atlas/normalized_microarray_donor10021/Probes.csv')
dx = pandas.concat([pandas.Series(cca_mod.x_loadings_[:,0]),
                   pandas.Series(cca_mod.x_loadings_[:,0])],
                   axis=1)
dx.loc[:,'symbol'] = probes.loc[:,'gene_symbol'].values
dx.loc[:,'name'] = probes.loc[:,'gene_name'].values

## PCA FIRST

In [ ]:
pcamod = PCA(n_components=100, random_state=123).fit(bigdf)
sum(pcamod.explained_variance_ratio_)

In [ ]:
pca_tfm = pandas.DataFrame(pcamod.transform(bigdf))

In [35]:
pca_tfm = pandas.DataFrame(np.load('/Users/jakevogel/Dropbox/McGill/Neur608/Final_Project/stuff/train_PCA.npz'
                                  )['arr_0'])

In [36]:
tr,te = train_test_split(pca_tfm, stratify=gdf.new_structure_acronym.values, random_state=123)
tr_y = gdf.loc[tr.index,['mni_nlin_y','mni_nlin_z']]
te_y = gdf.loc[te.index,['mni_nlin_y','mni_nlin_z']]

In [38]:
cca_mod = PLSRegression(n_components=20)
cca_mod.fit(tr,tr_y)

PLSRegression(copy=True, max_iter=500, n_components=20, scale=True, tol=1e-06)

## ADDING X

In [11]:
gdf.loc[:,'abs_mni_nlin_x'] = abs(gdf.mni_nlin_x.values)

NameError: name 'gdf' is not defined

In [12]:
tr,te = train_test_split(pca_tfm, stratify=gdf.new_structure_acronym.values, random_state=123)
tr_y = gdf.loc[tr.index,['mni_nlin_y','mni_nlin_z','abs_mni_nlin_x']]
te_y = gdf.loc[te.index,['mni_nlin_y','mni_nlin_z','abs_mni_nlin_x']]

NameError: name 'pca_tfm' is not defined

In [13]:
plotr = pandas.concat([pandas.Series(cca_mod.predict(te)[:,0], index = te_y.index), 
                       pandas.Series(cca_mod.predict(te)[:,1], index = te_y.index),
                       pandas.Series(cca_mod.predict(te)[:,2], index = te_y.index),
                       pandas.Series(te_y.loc[:,te_y.columns[0]]),
                       pandas.Series(te_y.loc[:,te_y.columns[1]]),
                       pandas.Series(te_y.loc[:,te_y.columns[2]]),
                       gdf.loc[te_y.index,'slab_type']], 
                      axis=1)
plotr.columns = ['predicted_y', 'predicted_z', 'predicted x',
                 'observed_y', 'observed_z', 'observed_x', 'slab_type']

SyntaxError: invalid syntax (<ipython-input-13-186986e08cb9>, line 4)

In [ ]:
for i in range(3):
    plt.close()
    sns.lmplot(plotr.columns[i], plotr.columns[i+3], data = plotr)
    plt.show()
    print(stats.pearsonr(cca_mod.predict(te)[:,i],te_y.loc[:,te_y.columns[i]]))

In [ ]:
for i in range(3):
    plt.close()
    sns.lmplot(plotr.columns[i], plotr.columns[i+3], col = 'slab_type', data = plotr)
    plt.show()

## less components

In [ ]:
cca_mod = PLSRegression(n_components=1)
cca_mod.fit(tr,tr_y)

Notes
================

* PCA first then PLS works plenty well and also very fast -- will support 10-fold CV (0.83, 0.89, 0.74)
* Keeping the same number of components as Ys leaves Ys orthogonal
* Y and Z are predicted well (r < 0.8), x not nearly as well (r ~ 0.5)
* Combination X,Y,Z, with 1 component seems to favor Z but cost all three (0.53, 0.80, 0.40)
* Combination X,Y,Z with 2 components maintains Y (0.8) and combines Z and X (0.82, 0.60).
* Forcing Z and Y together (no X) hurts both (0.63, 0.78)
* It seems 

In [14]:
np.random.permutation?

In [ ]:
def evaluate_components(clf, x, y, n_iterations=500, check = 100,
                       evaluate = True, plot = True, thr = 0.95):
    
    if type(x) != type(pandas.DataFrame()):
        x = pandas.DataFrame(x)
    
    # fit model
    clf.fit(x,y)
    n_comps = clf.n_components
    
    # prepare output
    results = pandas.DataFrame(index = range(n_comps * (n_iterations+1)),
                              columns = ['score', 'component', 'model'])
    results.loc[:,'component'] = list(range(n_comps))*(n_iterations+1)
    results.loc[range(n_comps),'model'] = ['True']*n_comps
    results.loc[range(n_comps,n_comps*(n_iterations+1)), 'model'
               ] = ['Null']*(n_comps*n_iterations)
    
    true_scores = [stats.pearsonr(clf.x_scores_[:,x], clf.y_scores_[:,x]
                                 )[0]**2 for x in range(n_comps)]
    results.loc[results[results.model=='True'].index,'score'] = true_scores
    
    k = clf.n_components
    # permute and refit model
    for i in range(n_iterations):
        new_ind = np.random.permutation(x.index)
        new_x = x.loc[new_ind]
        newmod = clf.fit(new_x,y)
        new_scores = [stats.pearsonr(newmod.x_scores_[:,x], newmod.y_scores_[:,x]
                                 )[0]**2 for x in range(n_comps)]
        results.loc[range(k, k+n_comps), 'score'] = new_scores
        if check:
            if i % check == 0:
                print('finished iteration',i)
        
        k += n_comps
        
    if evaluate:
        if plot:
            display_results(results, thr)
        else:
            display_results(results, thr, False)
        
    return results

def display_results(results, thr = 0.95, plot=True):
    
    if plot:
        # plot components
        sns.set_context('paper')
        plt.close()
        sns.factorplot(x='component', y = 'score', hue='model', data=results)
        plt.show()
    
    # get p-values
    for i in results.component.unique():
        nullz = results[(results.component==i) & (results.model=='Null')
                       ]['score'].sort_values().values
        real = results[(results.component==i) & (results.model=='True')]['score'].values[0]
        p = (len(nullz[nullz>real])+1) / len(nullz)
        if p < (1 - thr):
            print('component %s: p = %s  ***'%(i,p))
        else:
            print('component %s: p = %s'%(i,p))

In [15]:
def rotate_2d(clf, y, component = 0, return_rotations = False, mni_pth=None, vs=1):
    
    if type(y) == type(pandas.DataFrame()):
        y = y.values
    d1 = y[:,0]
    d2 = y[:,1]
    
    d1_wt = clf.y_rotations_[0, component]
    d2_wt = clf.y_rotations_[1, component]
    null = np.zeros(len(d1))
    
    d1_rot_x = d1 * np.cos(d1_wt) - null * np.sin(d1_wt)
    d1_rot_y = d1 * np.sin(d1_wt) + null * np.cos(d1_wt)
    d2_rot_x = null * np.cos(d2_wt) - d2 * np.sin(d2_wt)
    d2_rot_y = null * np.sin(d2_wt) + d2 * np.cos(d2_wt)
    
    plt.close()
    sns.regplot(x = d1, y = null, fit_reg=False, color = 'k')
    sns.regplot(x = null, y = d2, fit_reg=False, color = 'k')
    sns.regplot(x = d1_rot_x, y = d1_rot_y, fit_reg=False, label = 'new_d1')
    sns.regplot(x = d2_rot_x, y = d2_rot_y, fit_reg=False, label = 'new_d2')
    plt.legend()
    plt.show()
    
    if mni_pth:
        print('ORIGINAL AXES')
        mni_display(mni_pth, d1, null, null, d2, vs)
        print('ROTATED AXES')
        mni_display(mni_pth, d1_rot_x, d1_rot_y, d2_rot_x, d2_rot_y, vs)
    
    if return_rotations:
        return d1_rot_x, d1_rot_y, d2_rot_x, d2_rot_y
        
def mni_display(mni_pth, x1, x2, y1, y2, vs=1):
    
    mni = ni.load(mni_pth)
    axes = np.zeros_like(mni.get_data())
    
    iy = [convert_coords((0, x1[x], x2[x]), to_what='xyz', vs=vs) for x in range(len(x2))]
    iz = [convert_coords((0,  y1[x], y2[x]), to_what='xyz', vs=vs) for x in range(len(y2))]
    
    for coord in iy:
        coord = tuple([int(round(x)) for x in coord]) 
        axes[coord] = 1
    for coord in iz:
        coord = tuple([int(round(x)) for x in coord]) 
        axes[coord] = 2
    
    axes_img_null = ni.Nifti1Image(axes, affine=mni.affine)
    plt.close()
    plotting.plot_roi(axes_img_null, mni,  draw_cross=False)
    plt.show()

In [ ]:
def convert_coords(coord, to_what = 'mni', vs = 1):
    origin = [90, 126, 72]
    origin = (np.array(origin) / vs).astype(int).tolist()
    x,y,z = coord[0],coord[1],coord[2]

    if to_what == 'mni':
        x = (origin[0]-x)*vs
        y = (y-origin[1])*vs
        z = (z-origin[2])*vs

    elif to_what == 'xyz':
        x=origin[0]- x/vs
        y=y/vs + origin[1]
        z=z/vs + origin[2]

    else:
        raise IOError('please set to_what to either mni or xyz')

    return x,y,z

In [ ]:
convert_coords([0,0,0], to_what='xyz')

In [ ]:
mni = ni.load(MNI152_FILE_PATH)


## Features

In [39]:
cca_mod.x_loadings_.shape

(100, 20)

In [40]:
cca_mod.x_weights_.shape

(100, 20)

In [41]:
cca_mod.coef_.shape

(100, 2)

In [44]:
tr.shape

(2776, 100)

In [45]:
np.random.choice?

In [ ]:
def bootstrap_features(clf, x, y, n_iterations=500, check = 100, on ='x'):
    
    if type(x) != type(pandas.DataFrame()):
        x = pandas.DataFrame(x)
    if type(y) != type(pandas.DataFrame()):
        y = pandas.DataFrame(y)
    
    # fit model
    orig = clf.fit(x,y)
    
    # prepare output
    n_feats_x = x.shape[-1]
    n_feats_y = y.shape[-1]
    all_results_x = {}
    all_results_y = {}
    
    for i in range(orig.n_components):
        results = pandas.DataFrame(index = range(n_iterations), columns = range(n_feats_x))
        all_results_x.update({i: results})
        results = pandas.DataFrame(index = range(n_iterations), columns = range(n_feats_y))
        all_results_y.update({i: results})
    bs_ratio_x = pandas.DataFrame(index = range(orig.n_components), 
                                columns = range(n_feats_x))
    bs_ratio_y = pandas.DataFrame(index = range(orig.n_components), 
                                columns = range(n_feats_y))
    
    # bootstrap
    for i in range(n_iterations):
        n_ind = np.random.choice(x.index, len(x.index))
        n_samp = pandas.DataFrame(x.loc[n_ind],copy=True)
        newmod = clf.fit(n_samp,y)
        for c in range(orig.n_components):
            xcorrs = [stats.pearsonr(orig.x_loadings_[:,c],
                                     newmod.x_loadings_[:,x])[0]**2 for x in range(orig.n_components)]
            closest = np.argmax(xcorrs)
            all_results_x[c].loc[i] = newmod.x_loadings_[:,closest]
            all_results_y[c].loc[i] = newmod.y_loadings_[:,closest]
        if check:
            if i % check == 0:
                print('finished iteration',i)
    
    # calculate bootstrap ratio
    bs_ratio_x = bootstrap_ratio(all_results_x, bs_ratio_x)
    bs_ratio_y = bootstrap_ratio(all_results_y, bs_ratio_y)
    
    return bs_ratio_x, bs_ratio_y
    
def bootstrap_ratio(all_results, bs_ratio):

    for i,res in all_results.items():
        bs_ratio.loc[i] = res.mean() / res.sem()
        
    return bs_ratio

        

In [ ]:
def back_transform_fmri(weights, component_matrix)

    weight_tfm = np.dot(component_matrix.T, weights)
    mat = np.zeros((272,272))
    mat[np.tril_indices_from(mat,k=-1)] = weight_tfm
    i_upper = np.triu_indices_from(mat, -1)
    mat[i_upper] = mat.T[i_upper]

In [ ]:
cca_mod.n_components

In [ ]:
def back_transform